In [ ]:
# Core
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display
from IPython.display import display

# Settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

print("Environment ready.")


In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d avarice02/urinalysis-test-results


In [ ]:
!unzip urinalysis-test-results.zip


In [ ]:
# Inspect extracted files
!ls


In [ ]:
# Load CSV (adjust filename if Kaggle updates it)
df = pd.read_csv("urinalysis_test_results.csv")

print("Dataset loaded successfully.")
display(df.head())


In [ ]:
print("Dataset Shape:", df.shape)
print("\nData Types & Non-null Counts:")
df.info()

print("\nSummary Statistics:")
display(df.describe(include="all"))


In [ ]:
missing = df.isnull().sum().sort_values(ascending=False)
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    "Missing Count": missing,
    "Missing (%)": missing_pct
})

display(missing_df)


In [ ]:
plt.figure(figsize=(14,6))
sns.heatmap(df.isnull(), cbar=False)
plt.title("Missing Data Heatmap")
plt.show()


In [ ]:
ordinal_map = {
    "Negative": 0,
    "Trace": 1,
    "1+": 2,
    "2+": 3,
    "3+": 4,
    "4+": 5
}

ordinal_cols = [
    "Protein",
    "Glucose",
    "Ketones",
    "Leukocytes",
    "Blood"
]

for col in ordinal_cols:
    if col in df.columns:
        df[col] = df[col].map(ordinal_map)


In [ ]:
binary_map = {"Negative": 0, "Positive": 1}

binary_cols = ["Nitrite", "Bacteria", "Crystals"]

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map(binary_map)


In [ ]:
date_cols = [c for c in df.columns if "date" in c.lower()]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

print("Parsed date columns:", date_cols)


In [ ]:
if "pH" in df.columns:
    sns.histplot(df["pH"].dropna(), kde=True)
    plt.axvline(4.5, color="red", linestyle="--", label="Lower Normal")
    plt.axvline(8.0, color="red", linestyle="--", label="Upper Normal")
    plt.legend()
    plt.title("Urine pH Distribution")
    plt.show()


In [ ]:
if "Specific Gravity" in df.columns:
    sns.histplot(df["Specific Gravity"].dropna(), kde=True)
    plt.axvline(1.005, color="red", linestyle="--")
    plt.axvline(1.030, color="red", linestyle="--")
    plt.title("Specific Gravity Distribution")
    plt.show()


In [ ]:
abnormal_counts = {}

for col in ordinal_cols + binary_cols:
    if col in df.columns:
        abnormal_counts[col] = (df[col] > 0).sum()

abnormal_df = pd.DataFrame.from_dict(
    abnormal_counts, orient="index", columns=["Abnormal Count"]
)

display(abnormal_df.sort_values("Abnormal Count", ascending=False))


In [ ]:
abnormal_df.sort_values("Abnormal Count").plot(
    kind="barh",
    legend=False,
    title="Frequency of Abnormal Urinalysis Findings"
)
plt.show()


In [ ]:
def count_clinical_outliers(series, low, high):
    return ((series < low) | (series > high)).sum()

outliers = {}

if "pH" in df.columns:
    outliers["pH"] = count_clinical_outliers(df["pH"], 4.5, 8.0)

if "Specific Gravity" in df.columns:
    outliers["Specific Gravity"] = count_clinical_outliers(
        df["Specific Gravity"], 1.005, 1.030
    )

outliers_df = pd.DataFrame.from_dict(
    outliers, orient="index", columns=["Outlier Count"]
)

display(outliers_df)


In [ ]:
for col in ["pH", "Specific Gravity"]:
    if col in df.columns:
        sns.boxplot(x=df[col])
        plt.title(f"{col} Boxplot")
        plt.show()


In [ ]:
display(df.head(10))
print("Final shape:", df.shape)


In [ ]:
df.to_csv("urinalysis_cleaned.csv", index=False)
print("Cleaned dataset saved as urinalysis_cleaned.csv")
